# Day 8

We have a 7-segment digit display that displays four digits at a time. However, the segments and signals are broken. So for each four digit display, the correct number of segments light up but not the correct ones. Say that b/g light up, then it must be a 1 as it's the only digit with two segments, so b/g = c/f but we still dont know which one's c and which one's f!



In [ ]:
# Digits like so:
#   0:      1:      2:      3:      4:
#  aaaa    ....    aaaa    aaaa    ....
# b    c  .    c  .    c  .    c  b    c
# b    c  .    c  .    c  .    c  b    c
#  ....    ....    dddd    dddd    dddd
# e    f  .    f  e    .  .    f  .    f
# e    f  .    f  e    .  .    f  .    f
#  gggg    ....    gggg    gggg    ....
# 
#   5:      6:      7:      8:      9:
#  aaaa    aaaa    aaaa    aaaa    aaaa
# b    .  b    .  .    c  b    c  b    c
# b    .  b    .  .    c  b    c  b    c
#  dddd    dddd    ....    dddd    dddd
#.     f  e    f  .    f  e    f  .    f
#.     f  e    f  .    f  e    f  .    f
#  gggg    gggg    ....    gggg    gggg

#Rules
# 5 letters: 3,5 and 2. 6 letters: 0, 6 and 9
# 3: If 5 letters and contains 1, then it's a three.
# 9: If it's 6 letters and contains 3, then it's a nine.
# 5: If 5 letters and contains (4-1) then it's a five.
# 2: If 5 letters and is not 3 or 5.
# 0: If 6 letters and conntains 1 and is not 9.

In [130]:
data =  open('input_day8.txt','r').read().splitlines()
#Test data
#data = open('input_day8_test.txt','r').read().splitlines()
input_vals, output_vals = [],[]

for d in data:
    input, output = d.split(' | ')
    input_vals.append(input)
    output_vals.append(output)


In [11]:
# Task 1, only find the number of "easy digits" in the output values.

count = 0
for output in output_vals:
    for signal in output.split(' '):
        if len(signal) in [2,4,3,7]:
            count += 1

print(f'There are {count} occasions of the numbers 1,4,7 or 8 in the output values.')


There are 452 occasions of the numbers 1,4,7 or 8 in the output values.


In [128]:
import numpy as np

def compare_strings(signal1,signal2):
    """ Compares if the two strings contain the same charachters.
    
    """
    
    if len(signal1) != len(signal2):
        return False
    elif sorted(signal1) == sorted(signal2):
        return True
    return False

def find_easy(input_list,no):
    if no == 1:
        length = 2
    elif no == 4:
        length = 4
    elif no == 7:
        length = 3
    elif no == 8:
        length = 7
    else:
        raise KeyError(f'{no} is not a valid number to find with this funciton.')
    
    i = 0
    for input in input_list:
        if len(input) == length:
            return i
        i += 1
    
# 5 letters: 3,5 and 2. 6 letters: 0, 6 and 9
# 3: If 5 letters and contains 1, then it's a three.
def find_three(input_list:list[str],index_1:int):
    i = 0

    for input in input_list:
        if len(input) == 5 and all(char in input for char in input_list[index_1]):
            return i
        i += 1
    return -1

# 9: If it's 6 letters and contains 3, then it's a nine.
def find_nine(input_list:list[str],index_3:int):
    i = 0

    for input in input_list:
        if len(input) == 6 and all(char in input for char in input_list[index_3]):
            return i
        i += 1
    return -1


# 5: If 5 letters and contains (4-1) then it's a five.
def find_five(input_list:list[str],index_1:int,index_4):
    splice = input_list[index_4]

    for char in input_list[index_1]:
        splice = splice.replace(char,"")

    i = 0
    for input in input_list:
        if len(input) == 5 and all(char in input for char in splice):
            return i
        i += 1
    return -1

# 2: If 5 letters and is not 3 or 5.
def find_two(input_list:list[str],index_3:int,index_5):

    i = 0
    for input in input_list:
        if len(input) == 5 and input != input_list[index_3] and input != input_list[index_5]:
            return i
        i += 1
    return -1

# 0: If 6 letters and conntains 1 and is not 9.
def find_zero(input_list:list[str],index_1:int,index_9):

    i = 0
    for input in input_list:
        if  len(input) == 6 and all(char in input for char in input_list[index_1]) and input != input_list[index_9]:
            return i
        i += 1
    return -1

# 6: If 6 letters and not a nine or zero.
def find_six(input_list:list[str],index_0:int,index_9):

    i = 0
    for input in input_list:
        if len(input) == 6 and input != input_list[index_0] and input != input_list[index_9]:
            return i
        i += 1
    return -1

def find_segments(input: str):
    input_list = input.split(' ')

    indexes = [int]*10
    # Finding the 1,4,7 and 8
    indexes[1],indexes[4],indexes[7],indexes[8] = find_easy(input_list,1),find_easy(input_list,4),find_easy(input_list,7),find_easy(input_list,8)

    # Finding the 3
    indexes[3] = find_three(input_list,indexes[1])
    indexes[9] = find_nine(input_list,indexes[3])
    indexes[5] = find_five(input_list,indexes[1],indexes[4])
    indexes[2] = find_two(input_list,indexes[3],indexes[5])
    indexes[0] = find_zero(input_list,indexes[1],indexes[9])
    indexes[6] = find_six(input_list,indexes[0],indexes[9])

    for i in range(0,10):
        number = int
        k = 0
        for index in indexes:
            if index == i: 
                number = k
            k += 1

        #print(f'{input_list[i]} is equal to {number}')

    return indexes


def calculate_number(input:str,output:str,indexes:list[int]):
    input_list = input.split(' ')
    output_list = output.split(' ')
    output_numbers = []

    for output in output_list:
        number = int
        for i in range(0,10):
            if len(output) == len(input_list[i]) and all(char in output for char in input_list[i]):
                k = 0
                for index in indexes:
                    if index == i: 
                        number = k
                    k += 1
        
        output_numbers.append(number)

    return output_numbers[0]*1000+output_numbers[1]*100+output_numbers[2]*10+output_numbers[3]


    



In [129]:
test_input = 'acedgfb cdfbe gcdfa fbcad dab cefabd cdfgeb eafb cagedb ab'
test_output = 'cdfeb fcadb cdfeb cdbaf'

#find_segments(test_input)
calculate_number(test_input, test_output,find_segments(test_input))



5353

In [132]:
# Doing it for all numbers:
sum = 0
for i in range(0, len(input_vals)):
    sum += calculate_number(input_vals[i], output_vals[i],find_segments(input_vals[i]))
print(sum)

1096964
